In [82]:
from dotenv import load_dotenv
load_dotenv()

import json

with open("user_cv.json",'r') as file:
    user = json.load(file)

def get_compliation(system_message, user_input, api_key=None):
    from openai import OpenAI

    if not api_key:
        api_key = os.envirom['OPENAI_API_KEY']

    client = OpenAI(api_key=api_key)
    stream = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_input},
        ],
        stream=False,
    )
    return stream

In [74]:
prompt = f"""
Your goal is to complete the information missing in the user data.
Provide questions to resolve missing data, user data is:

{json.dumps(user,indent=4)}

Provide all questions in the following format:

```json
[{{
    "question":"<the question to the user>",
    "regex":"<a python regex to validate the user input>",
    "regex_fail_message":"<the message if the regex fail>",
    "target_missing": [xpath from the root of the json to the key in a list]
}},
// more if you have
]
```

"""
print(prompt)



Your goal is to complete the information missing in the user data.
Provide questions to resolve missing data, user data is:

{
    "personal_info": {
        "firstname": "Netanel",
        "lastname": "Erlich",
        "email": "erlichnet57@gmail.com",
        "phone": "+972-52-4306924",
        "address": "Tel-Aviv",
        "linkedin": {
            "link": "https://www.linkedin.com/in/anna-kuzin/",
            "username": []
        },
        "github": {
            "link": "https://github.com/EhrlichNati",
            "username": [
                "EhrlichNati"
            ]
        },
        "additional_websites": []
    },
    "education": [
        {
            "degree": "Student for Bioinformatics",
            "institution": "Tel Aviv University",
            "location": "Tel Aviv",
            "graduation_date": "Expected 2025",
            "grade": ""
        },
        {
            "degree": "Pre-Med and Biology",
            "institution": "Tel Aviv University",
    

In [75]:
response = get_compliation("",prompt)

In [76]:
print(response.choices[0].message.content)

```json
[
    {
        "question": "What is your grade for the degree in Bioinformatics at Tel Aviv University?",
        "regex": "^[A-Za-z0-9\\-\\s]+$",
        "regex_fail_message": "Please enter a valid grade.",
        "target_missing": ["education", 0, "grade"]
    },
    {
        "question": "What was your responsibility at Tel Aviv University - Ron Shamir Lab's as a Data Analyst & Engineering?",
        "regex": "^[A-Za-z0-9,\\-\\s]+$",
        "regex_fail_message": "Please enter a valid responsibility description.",
        "target_missing": ["experience", 0, "responsibilities"]
    },
    {
        "question": "Where was your position located as a Psychometric Essay Examiner?",
        "regex": "^[A-Za-z\\-\\s]+$",
        "regex_fail_message": "Please enter a valid location.",
        "target_missing": ["experience", 2, "location"]
    },
    {
        "question": "What was the name of the company you worked for as an Independent Psychometrics and Mathematics Tutor?",
    

In [77]:
question_to_ask = json.loads(response.choices[0].message.content.replace("```json","").replace("```",""))
question_to_ask

[{'question': 'What is your grade for the degree in Bioinformatics at Tel Aviv University?',
  'regex': '^[A-Za-z0-9\\-\\s]+$',
  'regex_fail_message': 'Please enter a valid grade.',
  'target_missing': ['education', 0, 'grade']},
 {'question': "What was your responsibility at Tel Aviv University - Ron Shamir Lab's as a Data Analyst & Engineering?",
  'regex': '^[A-Za-z0-9,\\-\\s]+$',
  'regex_fail_message': 'Please enter a valid responsibility description.',
  'target_missing': ['experience', 0, 'responsibilities']},
 {'question': 'Where was your position located as a Psychometric Essay Examiner?',
  'regex': '^[A-Za-z\\-\\s]+$',
  'regex_fail_message': 'Please enter a valid location.',
  'target_missing': ['experience', 2, 'location']},
 {'question': 'What was the name of the company you worked for as an Independent Psychometrics and Mathematics Tutor?',
  'regex': '^[A-Za-z\\-\\s]*$',
  'regex_fail_message': 'Please enter a valid company name.',
  'target_missing': ['experience', 3,

In [80]:
def set_value_at_xpath(current, xpath, value):
    for step in xpath[:-1]:
        if isinstance(current, list):
            current = current[step]
        elif isinstance(current, dict):
            current = current.get(step)
            if current is None:
                return
        else:
            return
    if isinstance(current, list):
        try:
            current[xpath[-1]] = value
        except IndexError:
            return
    elif isinstance(current, dict):
        current[xpath[-1]] = value

import re

def validate_regex(regex, string):
    try:
        re.compile(regex)
        match = re.search(regex, string)
        if match:
            return True
        else:
            return False
    except re.error:
        return False

for entry in question_to_ask[:1]:

    while True:
        print(f"Q: {entry['question']}")
        answer = input()
        if validate_regex(entry['regex'],answer):
            set_value_at_xpath(user,entry['target_missing'],answer)
            break
        else:
            print(entry['regex_fail_message'])

Q: What is your grade for the degree in Bioinformatics at Tel Aviv University?


 100


In [81]:
user

{'personal_info': {'firstname': 'Netanel',
  'lastname': 'Erlich',
  'email': 'erlichnet57@gmail.com',
  'phone': '+972-52-4306924',
  'address': 'Tel-Aviv',
  'linkedin': {'link': 'https://www.linkedin.com/in/anna-kuzin/',
   'username': []},
  'github': {'link': 'https://github.com/EhrlichNati',
   'username': ['EhrlichNati']},
  'additional_websites': []},
 'education': [{'degree': 'Student for Bioinformatics',
   'institution': 'Tel Aviv University',
   'location': 'Tel Aviv',
   'graduation_date': 'Expected 2025',
   'grade': '100'},
  {'degree': 'Pre-Med and Biology',
   'institution': 'Tel Aviv University',
   'location': 'Tel Aviv',
   'graduation_date': '2021',
   'grade': ''},
  {'degree': 'Full Matriculation',
   'institution': 'Kfar Ganim “Amit” High School',
   'location': 'Tel Aviv',
   'graduation_date': '2013',
   'grade': '5 units: Mathematics, English'}],
 'experience': [{'title': 'Data Analyst & Engeneering',
   'company': 'Tel Aviv University - Ron Shamir Lab’s',
  